In [ ]:
!py -m pip install pandas --user
!py -m pip install plotly --user
!py -m pip install ipykernel --user
!py -m pip install nbformat --user
!py -m pip install itables --user
!py -m pip install googlefinance --user

In [2]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dateutil.parser import parse
from itables import init_notebook_mode


Importing all data used for this notebook.

In [ ]:
init_notebook_mode(all_interactive=True)
covid_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", index_col=0, on_bad_lines='skip')
dowjones_data = pd.read_csv("data\Dow Jones Industrial Average Historical Data.csv")
m2real_data = pd.read_csv("data\M2REAL.csv")

Dropping unwanted columns and reshaping covid data from wide format to long format.

In [ ]:
us_data = covid_data[covid_data["Country/Region"] == "US"]
us_data = us_data.drop(columns = ["Lat","Long"])
us_data = us_data.melt("Country/Region", var_name='Date', value_name='Covid Cases')
us_data = us_data.drop(columns = "Country/Region")
us_data["Date"] = us_data["Date"].apply(lambda x: parse(str(x)).strftime("%Y-%m-%d"))
us_data


Retyping the dates of dowjones csv data to fit with covid data and later dictionary dates down in a lower code cell

In [7]:
dowjones_data["Date"] = dowjones_data["Date"].apply(lambda x: parse(str(x)).strftime("%Y-%m-%d"))
dowjones_data = dowjones_data.sort_values("Date",ascending=True)
dowjones_data["Price"] = dowjones_data["Price"].str.replace(",","").astype("float")

Quick column rename on m2real dataset

In [19]:
m2real_data = m2real_data.rename(columns={"DATE":"Date"})

5943.6

Combining covid, dowjones, and m2real data, filling any na values of the dowjones data with a forwardfill, which are a result of the stock market not being open during the weekends, with the price that was last seen on the friday before every weekend. We also fill up the na values of the m2 data using interpolation, because of the fact that the data is updated monthly from the source, we need to use linear interpolation to make it consistant with our dates.

In [ ]:
merged_data = us_data.merge(dowjones_data[["Date","Price"]], on=["Date"], how="left")
merged_data.ffill(inplace=True)
merged_data = merged_data.merge(m2real_data,on=["Date"],how="left")
# Performing this insert in order to line up January's M2 stock data with our dataset, as the M2 data is only monthly.
merged_data.iloc[0,3] = m2real_data.iloc[0,1]
# We need to interpolate the daily value due to our M2 data being only monthly
merged_data.interpolate(inplace=True)
merged_data

Now we graph our datasets using plotly, and create a dictionary of important dates of covid's pandemic timeline. We use this dictionary to create markers to add onto our plot.

In [ ]:

big_fig =  make_subplots(specs=[[{"secondary_y": True}]])

covid_fig = px.line(merged_data, x="Date", y=["Covid Cases"])
covid_fig.update_traces(line_color='#FF0000')

dowjones_fig = px.line(merged_data, x="Date", y=["Price"])
dowjones_fig.update_traces(yaxis="y2")

m2real_fig = px.line(merged_data, x="Date", y=['M2REAL'])
m2real_fig.update_traces(yaxis="y3", line_color="green",)
m2real_fig.update_layout(yaxis_visible=False, yaxis_showticklabels=False)


important_dates = {
    '2020-02-23': 'Italy becomes a global COVID-19 hotspot and issues Decree-law No. 6',
    '2020-03-03': 'The WHO declares COVID-19 a pandemic',
    '2020-03-15': 'U.S. Quarantine Shutdown begins.',
    '2020-03-27': 'CARES Act was signed into law.',
    '2020-04-03': 'The Paycheck Protection Program (PPP) enacted',
    '2020-04-27': 'Over 1 million (1,000,000) US confirmed cases.',
    '2020-04-30': 'U.S. launches Operation Warp Speed initiative to produce vaccinations.',
    '2020-05-09': 'U.S. unemployment rate is 14.7%, the highest it has ever been since the Great Depression.',
    '2020-06-10': "Over 2 million US confirmed cases.",
    '2020-07-07': 'Over 3 million US confirmed cases.',
    '2020-08-17': 'COVID-19 becomes 3rd leading cause of death in U.S.',
    '2020-12-11': 'FDA issues an EUA for the Pfizer-BioNTech COVID-19 vaccine.',
    '2020-12-18': 'FDA issues an EUA for the Moderna COVID-19 vaccine.',
    '2020-12-29': 'The first case of the COVID-19 B.1.1.7 / “Alpha” variant is detected in the U.S. by the Colorado Department of Health.',
    '2021-03-17': 'CDC announces $2.25 billion in spending over two years to address COVID-19 related health disparities for people living in rural areas and racial and ethnic minority groups.',
    '2021-05-17': 'An estimated 5.1 million women left the workforce when COVID-19 closed schools and child-care centers in 2020. Today, 1.3 million remain out of the workforce and only 56% of women in the U.S. are working for a salary the lowest percentage since 1986.',
    '2021-11-26': 'WHO designates the COVID-19 “Omicron” variant, first identified by scientists in South Africa, as a “variant of concern.”',
    '2021-12-01': 'The first case of the Omicron variant in the U.S. is detected by the California and San Francisco Departments of Public Health.',
    '2021-12-15': 'The recorded death toll from COVID-19 surpasses 800,000 in the U.S. One in every 100 people ages 65 years and older in the U.S. has died.',
    '2022-01-01': 'As Delta and Omicron spread, New York state records its highest number of new COVID-19 cases in a single day since the pandemic began with 114,082 new confirmed cases.',
    '2022-03-02': 'WHO releases data showing that the COVID-19 pandemic triggered a 25% increase in anxiety and depression worldwide, with young people and women at the highest risk.',
    '2022-05-12': 'The number of recorded deaths due to COVID-19 in the U.S. reaches 1 million (1,000,000).'
}

big_fig.add_traces(covid_fig.data + dowjones_fig.data)

for trace in m2real_fig.data:
    trace.yaxis = 'y3'
    big_fig.add_trace(trace)

# Layout for the third axis (y3)
big_fig.update_layout(
    yaxis3=dict(
        title="",
        overlaying="y",
        side="left",
        showgrid=False,
        showticklabels=False,
        position=0.05  
    ),
    yaxis=dict(
        title="COVID Cases"
    ),
    yaxis2=dict(
        title="Dow Jones Price"
    )
)
    
# big_fig.add_traces([covid_fig,dowjones_fig])
for date, event in important_dates.items():
    # Find the values for Covid Cases and Dow Jones Price on an important date
    covid_cases = merged_data[merged_data['Date'] == date]['Covid Cases'].values[0]

    dow_price = merged_data[merged_data['Date'] == date]['Price'].values[0]

    # m2real_stock = merged_data[merged_data['Date'] == date]['M2REAL'].values[0]

    # Add scatter plot for important dates for Covid Cases data line
    big_fig.add_trace(go.Scatter(x = [date], y = [covid_cases], mode = 'markers', marker = dict(color = 'yellow', size=10, line = dict(width = 2, color = 'DarkSlateGrey')),
                                name = event, hovertext = event, hoverinfo = 'text', showlegend=False))

    # Add same important dates for Dow Jones data line
    big_fig.add_trace(go.Scatter(x = [date], y = [dow_price], mode = 'markers', marker = dict(color = 'yellow', size = 10, line = dict(width = 2, color = 'DarkSlateGrey')),
                                name = event, hovertext = event, hoverinfo = 'text', yaxis = 'y2', showlegend=False))

    # And of course for the M2 data
    # big_fig.add_trace(go.Scatter(x = [date], y = [m2real_stock], mode = 'markers', marker = dict(color = 'yellow', size = 10, line = dict(width = 2, color = 'DarkSlateGrey')),
    #                             name = event, hovertext = event, hoverinfo = 'text', yaxis = 'y2', showlegend=False))


big_fig.show()

Sources:

Covid Data:

COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University https://github.com/CSSEGISandData/COVID-19

Covid Timeline info and dates:

https://www.cdc.gov/museum/timeline/covid19.html

Dowjones Data:

https://www.investing.com/indices/us-30-historical-data

M2 Real Money Stock:

Federal Reserve Bank of St. Louis, Real M2 Money Stock [M2REAL], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/M2REAL, December 19, 2023.
